In [1]:
import torchtext
from torchtext.data import Field, BucketIterator, TabularDataset
import spacy
import argparse
from utils_transform import *
from transformer import *
import pdb

In [2]:
class Args:
    def __init__(self):
        self.size= "mini"

In [3]:
params = Params(Args())

In [4]:
train, val, test, TR, EN = load_train_val_test_datasets(params)

Building vocab...
TR vocab size: 727, EN vocab size: 798
Done building vocab


In [5]:
pad_idx = EN.vocab.stoi[PAD]
train_iter = MyIterator(train, batch_size=512, device=torch.device('cpu'),
                        repeat=False, sort_key=lambda x: (len(x.src), len(x.trg), len(x.src_context)),
                        batch_size_fn=batch_size_fn, train=True)

In [6]:
bpemb_tr, bpemb_en = load_bpe(1000)

In [7]:
bpemb_en.decode(list(train.examples[1].src))

'babanızın▁da ihtiyacı var.'

In [8]:
bpemb_en.decode(list(train.examples[1].trg))

'your father has to use the bathroom too!'

In [9]:
train_csv, val_csv, test_csv = "train_mini.csv", "val_mini.csv", "test_mini.csv"

In [10]:
# DATA LOADING
en = spacy.load('en')
def tokenize_en(sentence):
    return [tok.text for tok in en.tokenizer(sentence)]

SOS, EOS, PAD, BOS = "<s>", "</s>", "<pad>", "<bos>" # Represents begining of context sentence
# Context and source / target fields for English + Turkish
TRF = Field(lower=True, pad_token=PAD)
ENF = Field(tokenize=tokenize_en, lower=True, pad_token=PAD, init_token = SOS, eos_token =EOS)


In [11]:
# Must be in order
data_fields = [
  ('src_context', TRF), ('src', TRF),
  ('trg_context', ENF), ('trg', ENF)]

train_old = TabularDataset.splits(
  path='data/', 
  train=train_csv,
  format='tsv', 
  fields=data_fields)[0]

In [12]:
train_old.examples[1].src

['babanızın', 'da', 'ihtiyacı', 'var.']

In [ ]:
for old_b in train_iter:
    new_b = rebatch(EN.vocab.stoi[PAD], old_b)
    pdb.set_trace()

> <ipython-input-14-431479b3b65e>(1)<module>()
-> for old_b in train_iter:
(Pdb) old_b

[torchtext.data.batch.Batch of size 30]
	[.src_context]:[torch.LongTensor of size 35x30]
	[.src]:[torch.LongTensor of size 14x30]
	[.trg_context]:[torch.LongTensor of size 43x30]
	[.trg]:[torch.LongTensor of size 17x30]
(Pdb) old_b.src
tensor([[ 68,  40,  52,   6,  60,  63, 256, 118,  68,  65, 250,  18, 115,  47,
           6, 336,   6, 413,   5,  56, 137,  25,  25, 116,   6,  60,   6,  56,
          94,  44],
        [103,  51,   5,  82,  11,  67,  10,   6, 103,  99,  43,  10,   7, 257,
          24,  97,  82, 266, 215, 244, 134, 198, 240,  20,  43,  93, 344,  87,
          21,  69],
        [ 66, 491,   2,  69,  85,  66,  10,  24,  66,   3,  34, 125,   6, 115,
          21, 266,  69, 237,  59,   9,   3, 216, 164,   8,   7, 107, 260, 136,
         488,  39],
        [258, 285,  29,  38,  12, 100,  30,  14, 125,  17,   6,  42,  71,   7,
         142,  58,  16, 235,  47, 491,  77, 282, 236,  26, 312,